In [1]:
#| include: false
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
cd(@__DIR__)

  Activating project at `~/gitrepos/kdheepak.github.io/blog/compile-time-vs-run-time-in-julia`


In [2]:
abstract type Shape end
area(::Shape) = 0.0

struct Square <: Shape
    side::Float64
end
area(s::Square) = s.side * s.side
    
struct Rectangle <: Shape
    width::Float64
    height::Float64
end
area(r::Rectangle) = r.width * r.height
    
struct Triangle <: Shape
    base::Float64
    height::Float64
end
area(t::Triangle) = 1.0/2.0 * t.base * t.height

struct Circle <: Shape
    radius::Float64
end
area(c::Circle) = π * c.radius^2

area (generic function with 5 methods)

In [3]:
using Test
@testset "Areas" begin
    @test area(Square(2)) == 4
    @test area(Rectangle(2,3)) == 6
    @test area(Triangle(2,3)) == 3
    @test area(Circle(2)) ≈ 4π
end;

Test Summary: | Pass  Total  Time
Areas         |    4      4  0.1s


In [4]:
using Random

Random.seed!(42)

function shape_builder(choice::Integer)
    if choice == 1
        Square(rand())
    elseif choice == 2
        Rectangle(rand(), rand())
    elseif choice == 3
        Triangle(rand(), rand())
    elseif choice == 4
        Circle(rand())
    end
end

count = 1_000_000
shapes = [shape_builder(rand((1,2,3,4))) for _ in 1:count];

In [5]:
#| echo: false
using Format
using Markdown
l = cfmt("%'d", length(shapes))
Markdown.md"The total number of shapes we have is $l."

The total number of shapes we have is 1,000,000.


In [6]:
typeof(shapes)

Vector{Shape} (alias for Array{Shape, 1})

In [7]:
main1(shapes) = sum(area.(shapes))

main1 (generic function with 1 method)

In [8]:
@time main1(shapes);

  0.130660 seconds (2.11 M allocations: 45.631 MiB, 44.62% gc time, 63.33% compilation time)


In [9]:
using BenchmarkTools

In [10]:
@benchmark main1(shapes)

BenchmarkTools.Trial: 119 samples with 1 evaluation.
 Range (min … max):  41.434 ms …  44.154 ms  ┊ GC (min … max): 0.00% … 1.43%
 Time  (median):     42.195 ms               ┊ GC (median):    1.17%
 Time  (mean ± σ):   42.231 ms ± 457.435 μs  ┊ GC (mean ± σ):  1.13% ± 0.54%

                 ▅▅▆▆▆▅█▂ ▃   ▃                                 
  ▅▄▇▇▄▅▄▁▅▇▁▇█▇▄██████████▅▁▄█▄▁▁▅▁▁▁▁▁▄▁▁▅▁▁▁▁▄▁▄▁▁▄▁▁▁▁▁▁▁▅ ▄
  41.4 ms         Histogram: frequency by time         43.8 ms <

 Memory estimate: 38.14 MiB, allocs estimate: 1999492.

In [11]:
shapes_by_type(::Type{T}, shapes) where T = [shape for shape in shapes if isa(shape, T)]

square_arr = shapes_by_type(Square, shapes)
rectangle_arr = shapes_by_type(Rectangle, shapes)
triangle_arr = shapes_by_type(Triangle, shapes)
circle_arr = shapes_by_type(Circle, shapes)

nothing

In [12]:
typeof(square_arr)

Vector{Square} (alias for Array{Square, 1})

In [13]:
typeof(rectangle_arr)

Vector{Rectangle} (alias for Array{Rectangle, 1})

In [14]:
typeof(triangle_arr)

Vector{Triangle} (alias for Array{Triangle, 1})

In [15]:
typeof(circle_arr)

Vector{Circle} (alias for Array{Circle, 1})

In [16]:
filter_shapes_by_type(::Type{T}, shapes) where T = filter(shapes) do s; isa(s, T); end

shape_arr1 = filter_shapes_by_type(Square, shapes)
shape_arr2 = filter_shapes_by_type(Rectangle, shapes)
shape_arr3 = filter_shapes_by_type(Triangle, shapes)
shape_arr4 = filter_shapes_by_type(Circle, shapes)

nothing

In [17]:
typeof(shape_arr1)

Vector{Shape} (alias for Array{Shape, 1})

In [18]:
typeof(shape_arr2)

Vector{Shape} (alias for Array{Shape, 1})

In [19]:
typeof(shape_arr3)

Vector{Shape} (alias for Array{Shape, 1})

In [20]:
typeof(shape_arr4)

Vector{Shape} (alias for Array{Shape, 1})

In [22]:
sorted_shapes_shape = vcat(square_arr, rectangle_arr, triangle_arr, circle_arr);
sorted_shapes_any = Any[s for s in sorted_shapes_shape];

In [23]:
typeof(sorted_shapes_shape)

Vector{Shape} (alias for Array{Shape, 1})

In [24]:
@benchmark main1(sorted_shapes_shape)

BenchmarkTools.Trial: 141 samples with 1 evaluation.
 Range (min … max):  33.594 ms … 38.594 ms  ┊ GC (min … max): 0.00% … 7.15%
 Time  (median):     34.695 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   35.533 ms ±  1.442 ms  ┊ GC (mean ± σ):  3.24% ± 3.38%

       ▆█ ▁                                                    
  ▄▁▁▃▄██▇█▅▄▃▄▃▃▁▁▁▃▁▃▃▁▁▃▁▁▁▃▁▁▁▃▃▁▃▁▆██▆▆▆▄▆▄▅▁▄▃▁▁▁▁▁▁▁▁▃ ▃
  33.6 ms         Histogram: frequency by time        38.4 ms <

 Memory estimate: 38.14 MiB, allocs estimate: 1999492.

In [25]:
typeof(sorted_shapes_any)

Vector{Any} (alias for Array{Any, 1})

In [26]:
@benchmark main1(sorted_shapes_any)

BenchmarkTools.Trial: 138 samples with 1 evaluation.
 Range (min … max):  33.935 ms … 39.610 ms  ┊ GC (min … max): 0.00% … 7.34%
 Time  (median):     35.934 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   36.247 ms ±  1.563 ms  ┊ GC (mean ± σ):  3.19% ± 3.36%

  ▄   ▂▄▆ ▄▄█      ▂▂    ▄          ▂  ▂▂▆▆▂       ▆ ▄         
  █▁▆▆███▆████▄▁██▆██▄▄▁▆█▁▁▆▁▁▁▁▁▆▆█▆▁█████▄▄▄▄▄▁▁█▆█▄▆█▄▄▄▆ ▄
  33.9 ms         Histogram: frequency by time          39 ms <

 Memory estimate: 38.14 MiB, allocs estimate: 1999492.

In [50]:
struct ShapeContainerSpecialized{T <: Shape}
    shape::T
end
area(s::ShapeContainerSpecialized) = area(s.shape)

area (generic function with 8 methods)

In [51]:
sorted_shapes_shapecontainer_specialized = [ShapeContainerSpecialized(s) for s in sorted_shapes_any];

In [52]:
typeof(sorted_shapes_shapecontainer_specialized)

Vector{ShapeContainerSpecialized} (alias for Array{ShapeContainerSpecialized, 1})

In [53]:
@time main1(sorted_shapes_shapecontainer_specialized);

  0.056874 seconds (1.00 M allocations: 23.077 MiB, 20.00% compilation time)


In [54]:
@benchmark main1(sorted_shapes_shapecontainer_specialized)

BenchmarkTools.Trial: 131 samples with 1 evaluation.
 Range (min … max):  36.206 ms … 46.573 ms  ┊ GC (min … max): 0.00% … 17.88%
 Time  (median):     37.124 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   38.268 ms ±  2.771 ms  ┊ GC (mean ± σ):  2.95% ±  5.87%

   █▆▁   ▃                                                     
  ████▅▆▅█▃▃▅▁▅▃▄▃▄▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▃▃▅▃▁▁▁▃▃▃▁▃▃▁▁▁▃ ▃
  36.2 ms         Histogram: frequency by time        46.3 ms <

 Memory estimate: 22.89 MiB, allocs estimate: 1000010.

In [32]:
struct ShapeContainerUntyped
    shape
end
area(s::ShapeContainerUntyped) = area(s.shape)

area (generic function with 7 methods)

In [33]:
sorted_shapes_shapecontainer_untyped = Any[ShapeContainerUntyped(s) for s in sorted_shapes_any];

In [34]:
@time main1(sorted_shapes_shapecontainer_untyped);

  0.085696 seconds (1.06 M allocations: 27.042 MiB, 36.33% compilation time: 45% of which was recompilation)


In [35]:
@benchmark main1(sorted_shapes_shapecontainer_untyped)

BenchmarkTools.Trial: 100 samples with 1 evaluation.
 Range (min … max):  48.312 ms … 56.302 ms  ┊ GC (min … max): 0.00% … 10.56%
 Time  (median):     48.980 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   50.076 ms ±  2.344 ms  ┊ GC (mean ± σ):  2.00% ±  3.90%

    ▁█▄▂ ▁                                                     
  ▃▃████▄█▇▃▁▃▄▃▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▃▃▃▁▃▆▃▁▃▁▁▃ ▃
  48.3 ms         Histogram: frequency by time        55.7 ms <

 Memory estimate: 22.89 MiB, allocs estimate: 1000010.

In [29]:
main2(arrs...) = sum(main1.(arrs))

main2 (generic function with 1 method)

In [48]:
any_arr1 = filter_shapes_by_type(Any, Square, shapes)
any_arr2 = filter_shapes_by_type(Any, Rectangle, shapes)
any_arr3 = filter_shapes_by_type(Any, Triangle, shapes)
any_arr4 = filter_shapes_by_type(Any, Circle, shapes)

@time main2(any_arr1, any_arr2, any_arr3, any_arr4);

  0.013918 seconds (1.00 M allocations: 22.890 MiB)


In [49]:
@benchmark main2(any_arr1, any_arr2, any_arr3, any_arr4)

BenchmarkTools.Trial: 355 samples with 1 evaluation.
 Range (min … max):  12.783 ms … 19.025 ms  ┊ GC (min … max): 0.00% … 30.33%
 Time  (median):     13.233 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.113 ms ±  1.724 ms  ┊ GC (mean ± σ):  6.31% ±  9.74%

    ▄█▇▆▄                                    ▁                 
  ▅▇██████▆▁▄▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▇▄▄▄█▅▇█████▇▇▆▁▆▆▁▁▁▁▄ ▆
  12.8 ms      Histogram: log(frequency) by time      18.6 ms <

 Memory estimate: 22.89 MiB, allocs estimate: 1000050.

In [33]:
@time main2(arr1, arr2, arr3, arr4);

  0.096021 seconds (466.29 k allocations: 38.885 MiB, 98.77% compilation time)


In [34]:
@benchmark main2(arr1, arr2, arr3, arr4)

BenchmarkTools.Trial: 4769 samples with 1 evaluation.
 Range (min … max):  484.958 μs …   8.573 ms  ┊ GC (min … max):  0.00% … 88.61%
 Time  (median):     856.042 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.044 ms ± 724.541 μs  ┊ GC (mean ± σ):  19.70% ± 20.74%

  ▃▃▃▃▄█▇▅▂▁                                                    ▁
  ██████████▇▅▁▁▁▁▁▁▁▁▁▅▅▄▅█▇█▇▅██▇▄▅▅▅▆▆▇▆▆▆▆▆▇▅▇▇▇▇▆▆▆▆▄▆▄▅▅▄ █
  485 μs        Histogram: log(frequency) by time       4.41 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 20.